In [0]:
# !pip install --upgrade tables
# !pip install eli5
# !pip install xgboost
# !pip install hyperopt

In [4]:
HOME = '/content/drive/My Drive/Colab Notebooks/matrix/dataworkshop_matrix_car'
%cd $HOME

/content/drive/My Drive/Colab Notebooks/matrix/dataworkshop_matrix_car


In [0]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

import xgboost as xgb

from hyperopt import hp, fmin, tpe, STATUS_OK

import warnings

In [0]:
df = pd.read_hdf('data/car.h5')

In [0]:
warnings.filterwarnings("ignore")

In [7]:
SUFFIX = '__cat'

for feat in df.columns:
    if isinstance(df[feat][0], list):
        continue

    factorized_value = df[feat].factorize()[0]
    if SUFFIX in feat:
        df[feat] = factorized_value
    else:
        df[feat + SUFFIX] = factorized_value

df.shape

(106494, 309)

In [0]:
def run_model(model, feats):
    X = df[feats]
    y = df['price_value']
    score = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
    return np.mean(score), np.std(score)

In [0]:
def map_rp(x):
    return -1 if str(x) == 'None' else int(x)


def map_m(x):
    return -1 if x == None else int(x.split(' ')[0])


def map_ps(x):
    return -1 if not isinstance(x, str) else int(x.split(' cm')[0].replace(' ', ''))


df['param_pojemność-skokowa'] = [map_ps(x) for x in df['param_pojemność-skokowa']]
df['param_moc'] = [map_m(x) for x in df['param_moc']]
df['param_rok-produkcji'] = [map_rp(x) for x in df['param_rok-produkcji']]

In [15]:
cat_feats = ['param_napęd__cat', 'param_rok-produkcji', 'param_stan__cat', 'param_skrzynia-biegów__cat', 'param_faktura-vat__cat', 'param_moc', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat', 'param_pojemność-skokowa', 'seller_name__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'param_wersja__cat', 'param_kod-silnika__cat', 'feature_system-start-stop__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_czujniki-parkowania-przednie__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_regulowane-zawieszenie__cat']
xgboost_param = {
    'max_depth': 5,
    'n_estimators': 50, 
    'learning_rate': 0.1,
    'seed': 0
}

xgboost_model = xgb.XGBRegressor(**xgboost_param)
run_model(xgboost_model, cat_feats)

[07:36:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:36:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:36:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)

In [0]:
def training(params):
    print('Trainning with:')
    print(params)

    mean_mea, _ = run_model(xgb.XGBRegressor(**params), cat_feats)
    return {'loss': np.abs(mean_mea), 'status': STATUS_OK}

In [0]:
xgboost_params = {
    'learning_rate': hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
    'max_depth': hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective': 'reg:squarederror',
    'n_estimators': 100,
    'seed': 0
}

In [25]:
best_ = fmin(training, xgboost_params, algo=tpe.suggest, max_evals=25)
best_

Trainning with:
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Trainning with:
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.05, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Trainning with:
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.25, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Trainning with:
{'colsample_bytree': 0.75, 'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8}
Trainning with:
{'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.05, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
Trainning with:
{'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.05, 'max_depth': 7, 'n_

{'colsample_bytree': 0.8, 'learning_rate': 1, 'max_depth': 8, 'subsample': 1.0}